# Parts 분리

In [21]:
import pandas as pd
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq
import matplotlib.pyplot as plt
import ast
from scipy.stats import ttest_ind
import seaborn as sns
import time
import networkx as nx
import warnings


# SettingWithCopyWarning 무시
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

# 모든 출력을 보여주도록 설정
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)


In [22]:
parquet_file = 'final_data.parquet'
# Parquet 파일 열기
table = pq.read_table(parquet_file)

# Table에서 데이터를 DataFrame으로 변환
df = table.to_pandas()
print(df.shape)

(13522573, 27)


In [23]:
# Print the column names
column_names = df.columns
print("Column Names:", column_names)

Column Names: Index(['MONTH', 'RECLAIM_PERIOD', 'SUBSIDIARY_NAME', 'CAPTIVE_CHANNEL_FLAG',
       'COMPANY_CODE', 'DIVISION_NAME', 'MODEL_CODE', 'PARTS_NO1', 'PARTS_NO2',
       'PARTS_NO3', 'PARTS_NO4', 'PARTS_NO5', 'RECEIPT_SYMPTOM', 'CENTER_CODE',
       'PREV_CENTER_CODE', 'PRODUCT_NAME', 'RECLAIM', 'WARRANTY_TYPE',
       'PREV_WARRANTY_TYPE', 'SVC_TYPE', 'PREV_SVC_TYPE', 'ENGINEER_CODE',
       'PREV_ENGINEER_CODE', 'ENGINEER_REPAIR', 'PREV_ENGINEER_SYMPTOM',
       'MULTICLAIM', 'PREV_RECEIPT_SYMPTOM'],
      dtype='object')


In [24]:
df.head(10)

,MONTH,RECLAIM_PERIOD,SUBSIDIARY_NAME,CAPTIVE_CHANNEL_FLAG,COMPANY_CODE,DIVISION_NAME,MODEL_CODE,PARTS_NO1,PARTS_NO2,PARTS_NO3,PARTS_NO4,PARTS_NO5,RECEIPT_SYMPTOM,CENTER_CODE,PREV_CENTER_CODE,PRODUCT_NAME,RECLAIM,WARRANTY_TYPE,PREV_WARRANTY_TYPE,SVC_TYPE,PREV_SVC_TYPE,ENGINEER_CODE,PREV_ENGINEER_CODE,ENGINEER_REPAIR,PREV_ENGINEER_SYMPTOM,MULTICLAIM,PREV_RECEIPT_SYMPTOM
0,01,47.0,LGEAI,Non Captive,HE,LTV,OLED65C7P,LGE_PARTS_993,LGE_PARTS_18111,LGE_PARTS_20442,None,None,None,LGE_CEN_0,LGE_CEN_0,OLED TV,1,IW,IW,IH,IH,LGE_ENG_0,LGE_ENG_0,Part replaced-electrical (ref#),Image Retention,2,Power
1,01,10.0,LGEAI,Non Captive,HE,LTV,65UN6955ZUF,LGE_PARTS_4492,None,None,None,None,Repair Status/Issue,LGE_CEN_1,LGE_CEN_1,LED LCD TV,1,IW,IW,IH,IH,LGE_ENG_1,LGE_ENG_1,Exchange-PC Board,No indication of power and totally inoperative...,2,Audio
3,01,39.0,LGEAI,Non Captive,HE,LTV,70UP7070PUE,LGE_PARTS_9199,None,None,None,None,None,LGE_CEN_3,LGE_CEN_3,LED LCD TV,1,IW,IW,IH,IH,LGE_ENG_3,LGE_ENG_3,Part replaced-electrical (ref#),No indication of power and totally inoperative...,2,Power
5,01,21.0,LGEAI,Non Captive,HE,LTV,50UN6955ZUF,LGE_PARTS_24647,LGE_PARTS_35272,None,None,None,None,LGE_CEN_3,LGE_CEN_3,LED LCD TV,1,IW,IW,IH,IH,LGE_ENG_3,LGE_ENG_3,Part replaced-electrical (ref#),No indication of power and totally inoperative...,2,Video
6,01,41.0,LGEAI,Non Captive,HE,LTV,55UN7300AUD,LGE_PARTS_5204,LGE_PARTS_21513,None,None,None,Network,LGE_CEN_0,LGE_CEN_0,LED LCD TV,1,IW,IW,IH,IH,LGE_ENG_0,LGE_ENG_0,Part replaced-electrical (ref#),No indication of power and totally inoperative...,2,Network
7,01,69.0,LGEAI,Non Captive,HE,LTV,65UK6500AUA,LGE_PARTS_6727,LGE_PARTS_9427,None,None,None,Video,LGE_CEN_5,LGE_CEN_3087,LED LCD TV,1,IW,IW,IH,IH,LGE_ENG_5,LGE_ENG_28700,Part replaced-electrical (ref#),No indication of power and totally inoperative...,2,Power
9,01,6.0,LGEAI,Non Captive,HE,LTV,OLED55CXPUA,LGE_PARTS_14364,None,None,None,None,Power,LGE_CEN_7,LGE_CEN_7,OLED TV,1,IW,IW,IH,IH,LGE_ENG_7,LGE_ENG_7,Part replaced-electrical (ref#),No indication of power and totally inoperative...,2,Power
10,01,22.0,LGEAI,Non Captive,HE,LTV,OLED55CXPUA,LGE_PARTS_20646,LGE_PARTS_28802,LGE_PARTS_22800,None,None,Power,LGE_CEN_8,LGE_CEN_811,OLED TV,1,IW,IW,IH,IH,LGE_ENG_8,LGE_ENG_1757,Part replaced-electrical (ref#),No indication of power and totally inoperative...,2,Power
11,01,35.0,LGEAI,Non Captive,HE,LTV,70UN7070PUA,LGE_PARTS_27662,LGE_PARTS_9611,None,None,None,Power,LGE_CEN_9,LGE_CEN_9,LED LCD TV,1,IW,IW,IH,IH,LGE_ENG_9,LGE_ENG_9,Part replaced-electrical (ref#),Intermittent display,2,None
12,01,44.0,LGEAI,Non Captive,HE,LTV,OLED55C1AUB,LGE_PARTS_35070,None,None,None,None,Connections,LGE_CEN_9,LGE_CEN_9,OLED TV,1,IW,IW,IH,IH,LGE_ENG_9,LGE_ENG_9,Part replaced-electrical (ref#),Intermittent display,2,Application


In [32]:
id_column = ['MONTH', 'RECLAIM_PERIOD', 'SUBSIDIARY_NAME', 'CAPTIVE_CHANNEL_FLAG',
       'COMPANY_CODE', 'DIVISION_NAME', 'MODEL_CODE', 'RECEIPT_SYMPTOM', 'CENTER_CODE',
       'PREV_CENTER_CODE', 'PRODUCT_NAME', 'RECLAIM', 'WARRANTY_TYPE',
       'PREV_WARRANTY_TYPE', 'SVC_TYPE', 'PREV_SVC_TYPE', 'ENGINEER_CODE',
       'PREV_ENGINEER_CODE', 'ENGINEER_REPAIR', 'PREV_ENGINEER_SYMPTOM',
       'MULTICLAIM', 'PREV_RECEIPT_SYMPTOM']

In [33]:
parts_column = ['PARTS_NO1', 'PARTS_NO2', 'PARTS_NO3', 'PARTS_NO4', 'PARTS_NO5']

In [34]:
# Melt the DataFrame to convert it from wide to long format
df_melted = pd.melt(df, id_vars=id_column, value_vars=parts_column, var_name='PART', value_name='VALUE')

In [35]:
# Drop rows where 'value' is None
df_melted = df_melted.dropna(subset=['VALUE'])

In [36]:
print(df_melted.shape)

(6650959, 24)


In [37]:
df_melted.head(10)

,MONTH,RECLAIM_PERIOD,SUBSIDIARY_NAME,CAPTIVE_CHANNEL_FLAG,COMPANY_CODE,DIVISION_NAME,MODEL_CODE,RECEIPT_SYMPTOM,CENTER_CODE,PREV_CENTER_CODE,PRODUCT_NAME,RECLAIM,WARRANTY_TYPE,PREV_WARRANTY_TYPE,SVC_TYPE,PREV_SVC_TYPE,ENGINEER_CODE,PREV_ENGINEER_CODE,ENGINEER_REPAIR,PREV_ENGINEER_SYMPTOM,MULTICLAIM,PREV_RECEIPT_SYMPTOM,PART,VALUE
0,01,47.0,LGEAI,Non Captive,HE,LTV,OLED65C7P,None,LGE_CEN_0,LGE_CEN_0,OLED TV,1,IW,IW,IH,IH,LGE_ENG_0,LGE_ENG_0,Part replaced-electrical (ref#),Image Retention,2,Power,PARTS_NO1,LGE_PARTS_993
1,01,10.0,LGEAI,Non Captive,HE,LTV,65UN6955ZUF,Repair Status/Issue,LGE_CEN_1,LGE_CEN_1,LED LCD TV,1,IW,IW,IH,IH,LGE_ENG_1,LGE_ENG_1,Exchange-PC Board,No indication of power and totally inoperative...,2,Audio,PARTS_NO1,LGE_PARTS_4492
2,01,39.0,LGEAI,Non Captive,HE,LTV,70UP7070PUE,None,LGE_CEN_3,LGE_CEN_3,LED LCD TV,1,IW,IW,IH,IH,LGE_ENG_3,LGE_ENG_3,Part replaced-electrical (ref#),No indication of power and totally inoperative...,2,Power,PARTS_NO1,LGE_PARTS_9199
3,01,21.0,LGEAI,Non Captive,HE,LTV,50UN6955ZUF,None,LGE_CEN_3,LGE_CEN_3,LED LCD TV,1,IW,IW,IH,IH,LGE_ENG_3,LGE_ENG_3,Part replaced-electrical (ref#),No indication of power and totally inoperative...,2,Video,PARTS_NO1,LGE_PARTS_24647
4,01,41.0,LGEAI,Non Captive,HE,LTV,55UN7300AUD,Network,LGE_CEN_0,LGE_CEN_0,LED LCD TV,1,IW,IW,IH,IH,LGE_ENG_0,LGE_ENG_0,Part replaced-electrical (ref#),No indication of power and totally inoperative...,2,Network,PARTS_NO1,LGE_PARTS_5204
5,01,69.0,LGEAI,Non Captive,HE,LTV,65UK6500AUA,Video,LGE_CEN_5,LGE_CEN_3087,LED LCD TV,1,IW,IW,IH,IH,LGE_ENG_5,LGE_ENG_28700,Part replaced-electrical (ref#),No indication of power and totally inoperative...,2,Power,PARTS_NO1,LGE_PARTS_6727
6,01,6.0,LGEAI,Non Captive,HE,LTV,OLED55CXPUA,Power,LGE_CEN_7,LGE_CEN_7,OLED TV,1,IW,IW,IH,IH,LGE_ENG_7,LGE_ENG_7,Part replaced-electrical (ref#),No indication of power and totally inoperative...,2,Power,PARTS_NO1,LGE_PARTS_14364
7,01,22.0,LGEAI,Non Captive,HE,LTV,OLED55CXPUA,Power,LGE_CEN_8,LGE_CEN_811,OLED TV,1,IW,IW,IH,IH,LGE_ENG_8,LGE_ENG_1757,Part replaced-electrical (ref#),No indication of power and totally inoperative...,2,Power,PARTS_NO1,LGE_PARTS_20646
8,01,35.0,LGEAI,Non Captive,HE,LTV,70UN7070PUA,Power,LGE_CEN_9,LGE_CEN_9,LED LCD TV,1,IW,IW,IH,IH,LGE_ENG_9,LGE_ENG_9,Part replaced-electrical (ref#),Intermittent display,2,None,PARTS_NO1,LGE_PARTS_27662
9,01,44.0,LGEAI,Non Captive,HE,LTV,OLED55C1AUB,Connections,LGE_CEN_9,LGE_CEN_9,OLED TV,1,IW,IW,IH,IH,LGE_ENG_9,LGE_ENG_9,Part replaced-electrical (ref#),Intermittent display,2,Application,PARTS_NO1,LGE_PARTS_35070


In [38]:
df_result = df_melted.drop(columns=['PART'])

In [40]:
df_result = df_result.rename(columns={'VALUE': 'PARTS_NO1'})

In [42]:
df_result.head(10)

,MONTH,RECLAIM_PERIOD,SUBSIDIARY_NAME,CAPTIVE_CHANNEL_FLAG,COMPANY_CODE,DIVISION_NAME,MODEL_CODE,RECEIPT_SYMPTOM,CENTER_CODE,PREV_CENTER_CODE,PRODUCT_NAME,RECLAIM,WARRANTY_TYPE,PREV_WARRANTY_TYPE,SVC_TYPE,PREV_SVC_TYPE,ENGINEER_CODE,PREV_ENGINEER_CODE,ENGINEER_REPAIR,PREV_ENGINEER_SYMPTOM,MULTICLAIM,PREV_RECEIPT_SYMPTOM,PARTS_NO1
0,01,47.0,LGEAI,Non Captive,HE,LTV,OLED65C7P,None,LGE_CEN_0,LGE_CEN_0,OLED TV,1,IW,IW,IH,IH,LGE_ENG_0,LGE_ENG_0,Part replaced-electrical (ref#),Image Retention,2,Power,LGE_PARTS_993
1,01,10.0,LGEAI,Non Captive,HE,LTV,65UN6955ZUF,Repair Status/Issue,LGE_CEN_1,LGE_CEN_1,LED LCD TV,1,IW,IW,IH,IH,LGE_ENG_1,LGE_ENG_1,Exchange-PC Board,No indication of power and totally inoperative...,2,Audio,LGE_PARTS_4492
2,01,39.0,LGEAI,Non Captive,HE,LTV,70UP7070PUE,None,LGE_CEN_3,LGE_CEN_3,LED LCD TV,1,IW,IW,IH,IH,LGE_ENG_3,LGE_ENG_3,Part replaced-electrical (ref#),No indication of power and totally inoperative...,2,Power,LGE_PARTS_9199
3,01,21.0,LGEAI,Non Captive,HE,LTV,50UN6955ZUF,None,LGE_CEN_3,LGE_CEN_3,LED LCD TV,1,IW,IW,IH,IH,LGE_ENG_3,LGE_ENG_3,Part replaced-electrical (ref#),No indication of power and totally inoperative...,2,Video,LGE_PARTS_24647
4,01,41.0,LGEAI,Non Captive,HE,LTV,55UN7300AUD,Network,LGE_CEN_0,LGE_CEN_0,LED LCD TV,1,IW,IW,IH,IH,LGE_ENG_0,LGE_ENG_0,Part replaced-electrical (ref#),No indication of power and totally inoperative...,2,Network,LGE_PARTS_5204
5,01,69.0,LGEAI,Non Captive,HE,LTV,65UK6500AUA,Video,LGE_CEN_5,LGE_CEN_3087,LED LCD TV,1,IW,IW,IH,IH,LGE_ENG_5,LGE_ENG_28700,Part replaced-electrical (ref#),No indication of power and totally inoperative...,2,Power,LGE_PARTS_6727
6,01,6.0,LGEAI,Non Captive,HE,LTV,OLED55CXPUA,Power,LGE_CEN_7,LGE_CEN_7,OLED TV,1,IW,IW,IH,IH,LGE_ENG_7,LGE_ENG_7,Part replaced-electrical (ref#),No indication of power and totally inoperative...,2,Power,LGE_PARTS_14364
7,01,22.0,LGEAI,Non Captive,HE,LTV,OLED55CXPUA,Power,LGE_CEN_8,LGE_CEN_811,OLED TV,1,IW,IW,IH,IH,LGE_ENG_8,LGE_ENG_1757,Part replaced-electrical (ref#),No indication of power and totally inoperative...,2,Power,LGE_PARTS_20646
8,01,35.0,LGEAI,Non Captive,HE,LTV,70UN7070PUA,Power,LGE_CEN_9,LGE_CEN_9,LED LCD TV,1,IW,IW,IH,IH,LGE_ENG_9,LGE_ENG_9,Part replaced-electrical (ref#),Intermittent display,2,None,LGE_PARTS_27662
9,01,44.0,LGEAI,Non Captive,HE,LTV,OLED55C1AUB,Connections,LGE_CEN_9,LGE_CEN_9,OLED TV,1,IW,IW,IH,IH,LGE_ENG_9,LGE_ENG_9,Part replaced-electrical (ref#),Intermittent display,2,Application,LGE_PARTS_35070


In [43]:
df_result.to_parquet('stage2_data.parquet', index=False)